<a href="https://colab.research.google.com/github/carlospessin/carlospessin-DIO-Bootcamp-Treinamento-de-Redes-Neurais-com-Transfer-Learning/blob/main/Projeto_de_Transfer_Learning_em_Python_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import zipfile
import os
import shutil
from PIL import Image
import numpy as np

In [ ]:
BATCH_SIZE = 32
IMG_SIZE = (224, 224)
EPOCHS = 10
DATASET_URL = "cats_vs_dogs.zip https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip"

In [ ]:
!wget -O cats_vs_dogs.zip {DATASET_URL}
with zipfile.ZipFile('cats_vs_dogs.zip', 'r') as zip_ref:
    zip_ref.extractall('cats_vs_dogs')

--2024-12-20 13:25:19--  http://cats_vs_dogs.zip/
Resolving cats_vs_dogs.zip (cats_vs_dogs.zip)... failed: Name or service not known.
wget: unable to resolve host address ‘cats_vs_dogs.zip’
--2024-12-20 13:25:19--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip
Resolving download.microsoft.com (download.microsoft.com)... 184.29.22.44, 2600:1406:5600:78b::317f, 2600:1406:5600:790::317f
Connecting to download.microsoft.com (download.microsoft.com)|184.29.22.44|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824887076 (787M) [application/octet-stream]
Saving to: ‘cats_vs_dogs.zip’

cats_vs_dogs.zip    100%[===================>] 786.67M   166MB/s    in 5.0s    

2024-12-20 13:25:25 (156 MB/s) - ‘cats_vs_dogs.zip’ saved [824887076/824887076]

FINISHED --2024-12-20 13:25:25--
Total wall clock time: 5.3s
Downloaded: 1 files, 787M in 5.0s (156 MB/s)


In [ ]:
base_dir = 'cats_vs_dogs/PetImages'
train_dir = os.path.join('cats_vs_dogs_data', 'train')
val_dir = os.path.join('cats_vs_dogs_data', 'val')

if not os.path.exists(train_dir):
    os.makedirs(train_dir)
if not os.path.exists(val_dir):
    os.makedirs(val_dir)

for category in ['Cat', 'Dog']:
    os.makedirs(os.path.join(train_dir, category), exist_ok=True)
    os.makedirs(os.path.join(val_dir, category), exist_ok=True)

In [ ]:
for category in ['Cat', 'Dog']:
    img_path = os.path.join(base_dir, category)
    if not os.path.exists(img_path):
        print(f"Diretório {img_path} não encontrado. Verifique a extração do dataset.")
        continue

    images = os.listdir(img_path)
    images = [img for img in images if img.endswith(('.jpg', '.jpeg', '.png', 'webp'))]

    train_images = images[:int(len(images) * 0.8)]
    val_images = images[int(len(images) * 0.8):]

    for img in train_images:
        shutil.copy(os.path.join(img_path, img), os.path.join(train_dir, category, img))
    for img in val_images:
        shutil.copy(os.path.join(img_path, img), os.path.join(val_dir, category, img))

In [ ]:
def validate_images(directory):
    for root, _, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            try:
                with Image.open(file_path) as img:
                    img.verify()  # Verifica a integridade do arquivo
            except (IOError, SyntaxError):
                print(f"Removendo arquivo inválido: {file_path}")
                os.remove(file_path)

# Validar imagens nos diretórios de treino e validação
validate_images(train_dir)
validate_images(val_dir)

Removendo arquivo inválido: cats_vs_dogs_data/train/Dog/11702.jpg
Removendo arquivo inválido: cats_vs_dogs_data/train/Cat/666.jpg


/usr/local/lib/python3.10/dist-packages/PIL/TiffImagePlugin.py:935: UserWarning: Truncated File Read
  warnings.warn(str(msg))


In [ ]:
validate_images(train_dir)
validate_images(val_dir)

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [ ]:
val_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

Found 19998 images belonging to 2 classes.


In [ ]:
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

Found 5000 images belonging to 2 classes.


In [ ]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=IMG_SIZE + (3,))
base_model.trainable = False

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(50, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

In [ ]:
for layer in model.layers[:156]:
  layer.trainable=False
for layer in model.layers[156:]:
  layer.tainable=True

In [ ]:
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=val_generator,
    validation_steps=val_generator.samples // BATCH_SIZE,
    epochs=EPOCHS
)

Epoch 1/10
624/624 ━━━━━━━━━━━━━━━━━━━━ 1304s 2s/step - accuracy: 0.9168 - loss: 0.2321 - val_accuracy: 0.9760 - val_loss: 0.0656
Epoch 2/10
  1/624 ━━━━━━━━━━━━━━━━━━━━ 12:07 1s/step - accuracy: 0.9375 - loss: 0.2085

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


624/624 ━━━━━━━━━━━━━━━━━━━━ 2s 649us/step - accuracy: 0.9375 - loss: 0.2085 - val_accuracy: 1.0000 - val_loss: 0.0066
Epoch 3/10
624/624 ━━━━━━━━━━━━━━━━━━━━ 1299s 2s/step - accuracy: 0.9687 - loss: 0.0846 - val_accuracy: 0.9784 - val_loss: 0.0568
Epoch 4/10
624/624 ━━━━━━━━━━━━━━━━━━━━ 1s 498us/step - accuracy: 1.0000 - loss: 0.0195 - val_accuracy: 0.6250 - val_loss: 0.4873
Epoch 5/10
624/624 ━━━━━━━━━━━━━━━━━━━━ 1316s 2s/step - accuracy: 0.9694 - loss: 0.0787 - val_accuracy: 0.9802 - val_loss: 0.0518
Epoch 6/10
624/624 ━━━━━━━━━━━━━━━━━━━━ 1s 448us/step - accuracy: 0.9375 - loss: 0.1738 - val_accuracy: 1.0000 - val_loss: 0.0013
Epoch 7/10
624/624 ━━━━━━━━━━━━━━━━━━━━ 1267s 2s/step - accuracy: 0.9696 - loss: 0.0771 - val_accuracy: 0.9820 - val_loss: 0.0492
Epoch 8/10
624/624 ━━━━━━━━━━━━━━━━━━━━ 1s 458us/step - accuracy: 0.9688 - loss: 0.1589 - val_accuracy: 1.0000 - val_loss: 0.0112
Epoch 9/10
624/624 ━━━━━━━━━━━━━━━━━━━━ 1290s 2s/step - accuracy: 0.9723 - loss: 0.0715 - val_accurac

In [ ]:
loss, accuracy = model.evaluate(val_generator)
print(f"Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")

157/157 ━━━━━━━━━━━━━━━━━━━━ 214s 1s/step - accuracy: 0.9773 - loss: 0.0587
Loss: 0.0634, Accuracy: 0.9772


In [ ]:
model.save('cats_vs_dogs_transfer_learning.h5')

In [ ]:
def predict_image(image_path):
    img = load_img(image_path, target_size=(224, 224))  # Redimensionar a imagem
    img_array = img_to_array(img)  # Converter para array NumPy
    img_array = np.expand_dims(img_array, axis=0)  # Adicionar dimensão de lote
    img_array = img_array / 255.0  # Normalizar os pixels

    prediction = model.predict(img_array)

    if prediction[0][0] > 0.5:
        return f"A imagem é de um cachorro com {prediction[0][0] * 100:.2f}% de confiança."
    else:
        return f"A imagem é de um gato com {(1 - prediction[0][0]) * 100:.2f}% de confiança."

In [ ]:
image_path = 'imagem.webp'  # Substitua pelo caminho da imagem
result = predict_image(image_path)
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
A imagem é de um cachorro com 86.65% de confiança.
